In [16]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score, confusion_matrix ,roc_curve,roc_auc_score
from tqdm import tqdm
from PIL import Image

In [30]:
train_path = 'planttraits2024/train.csv'
test_path = 'planttraits2024/test.csv'
MAX_EPOCH = 5
BATCH_SIZE = 1
weight_decay = 1e-3
LR=0.0001
img_size = 128
sd_features = ['X4_sd', 'X11_sd','X18_sd', 'X26_sd', 'X50_sd', 'X3112_sd']
label_features = ['X4_mean', 'X11_mean', 'X18_mean', 'X26_mean', 'X50_mean', 'X3112_mean']


In [144]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
train_df['id'] = train_df['id'].map(lambda x: 'train_images/' + str(x)+ ".jpeg")
test_df['id'] = test_df['id'].map(lambda x: 'test_images/' + str(x)+ ".jpeg")
FEATURE_COLS = test_df.columns[1:].tolist()

FileNotFoundError: [Errno 2] No such file or directory: 'planttraits2024/test.csv'

**Test sd?**

### Delete abnormal values

In [138]:
quntilelist=np.array(train_df[label_features].quantile([0,0.99]))
IQR=quntilelist[1]-quntilelist[0]
low_bound=quntilelist[0]-1.5*IQR
upper_bound=quntilelist[1]+1.5*IQR

for i in range(len(label_features)):
    train_df[label_features[i]] = np.where((train_df[label_features[i]]<upper_bound[i])&(train_df[label_features[i]]>low_bound[i]),\
                                           train_df[label_features[i]],-1)
for i in range(len(label_features)):
    train_df = train_df[train_df[label_features[i]]!=-1]

In [139]:
label_df = train_df[label_features+sd_features]
x_train,x_val,y_train,y_val = train_test_split(train_df, label_df, test_size=0.3)
x_train_images = np.array(x_train['id']) 
x_train = x_train.drop(columns = 'id')
x_val_images = np.array(x_val['id']) 
x_val = x_val.drop(columns = 'id')
y_train,y_val = np.array(y_train).astype(np.float32), np.array(y_val).astype(np.float32)

In [140]:
y_train[np.isnan(y_train)] = -1
y_train.max()

19305.188

In [141]:
scaler = RobustScaler()
train_features = scaler.fit_transform(x_train[FEATURE_COLS].values).astype(np.float32)
valid_features = scaler.transform(x_val[FEATURE_COLS].values).astype(np.float32)

scaler_y = RobustScaler()
train_labels = y_train
y_train = scaler_y.fit_transform(y_train)
y_val = scaler_y.transform(y_val)

train_labels = y_train[:,:6]
train_labels_aux = y_train[:,6:]
val_labels = y_val[:,:6]
val_labels_aux = y_val[:,6:]

train_features[np.isnan(train_features)] = -1
valid_features[np.isnan(valid_features)] = -1
train_labels[np.isnan(train_labels)] = -1
train_labels_aux[np.isnan(train_labels_aux)] = -1
val_labels[np.isnan(val_labels)] = -1
val_labels_aux[np.isnan(val_labels_aux)] = -1

In [143]:
train_labels.max()

41.721268

In [105]:
train_dataset = [(x_train_images[i], train_features[i],train_labels[i], train_labels_aux[i] ) for i in range(len(x_train_images))]
val_dataset = [(x_val_images[i], valid_features[i],val_labels[i],val_labels_aux[i]) for i in range(len(x_val_images))]
train_loader = DataLoader(dataset = train_dataset[:200], batch_size=BATCH_SIZE , shuffle= True, num_workers = 0)
val_loader = DataLoader(dataset = val_dataset, batch_size=BATCH_SIZE , shuffle= False, num_workers = 0)

In [58]:
class MLP(nn.Module):
    def __init__(self) -> None:
        super(MLP,self).__init__()
        self.linear1 = nn.Sequential(nn.Linear(163, 326), nn.SELU(),nn.Dropout())
        self.linear2 = nn.Sequential(nn.Linear(326, 64), nn.SELU(),nn.Dropout())
        self.dropout = nn.Dropout()
    def forward(self, x):
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.dropout(x)
        return x

class Combine_model(nn.Module):
    def __init__(self) -> None:
        super(Combine_model,self).__init__()
        self.model1 = models.resnet18(pretrained=False)
        self.model1.fc=nn.Sequential(
            nn.Linear(self.model1.fc.in_features, 128), 
            nn.Tanh()
        )
        self.model2 = MLP()
        self.fc = nn.Linear(128+64, 6)
        self.out_fc1 = nn.Linear(128+64,6)
        self.out_fc2 = nn.Sequential(
            nn.Linear(128+64,6), 
            nn.ReLU()
        )

    def forward(self,x,y):
        x = self.model1(x)
        y = self.model2(y)
        x = torch.cat([x,y], dim = 1)
        out1 = self.out_fc1(x)
        out2 = self.out_fc2(x)
        
        return out1, out2


In [51]:
train_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomResizedCrop(size=img_size, scale=(0.8, 1.0), ratio=(0.8, 1.25)),
        transforms.RandomRotation((-45,45)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.18,0.18,0.18], std=[0.24,0.24,0.24])
    ])
valid_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.18,0.18,0.18], std=[0.24,0.24,0.24])
])  


In [13]:
import ssl
model = Combine_model()
ssl._create_default_https_context = ssl._create_unverified_context

In [110]:
class R2Loss(nn.Module):
    def __init__(self, use_mask=False):
        super(R2Loss, self).__init__()
        self.use_mask = use_mask

    def forward(self, y_true, y_pred):
        if self.use_mask:
            mask = (y_true != -1)
            y_true = torch.where(mask, y_true, torch.zeros_like(y_true))
            y_pred = torch.where(mask, y_pred, torch.zeros_like(y_pred))
        SS_res = torch.sum((y_true - y_pred) ** 2,axis=0)
        SS_tot = torch.sum((y_true - torch.mean(y_true)) ** 2,axis=0)
        r2_loss = SS_res / (SS_tot + 1e-6)
        return torch.mean(r2_loss)
    
class MSELoss(nn.Module):
    def __init__(self, use_mask=False):
        super(MSELoss, self).__init__()
        self.use_mask = use_mask

    def forward(self, y_true, y_pred):
        if self.use_mask:
            mask = (y_true != -1)
            y_true = torch.where(mask, y_true, torch.zeros_like(y_true))
            y_pred = torch.where(mask, y_pred, torch.zeros_like(y_pred))

        mse_loss = torch.mean((y_true - y_pred) ** 2)
        return mse_loss

In [149]:
def train_epoch(train_loader,model, criterion1, criterion2, optimizer, scheduler, device, img_size):
    model.train()
    total_loss = 0
    for i, data in tqdm(enumerate(train_loader)):
        # Unpack data
        img_path, inputs, labels, labels_aux = data

        # Prepare image paths
        img_path = ['/Users/zhangyumeng/Desktop/23Winter/273P/project/planttraits2024/'+i for i in img_path]

        # # Prepare inputs and labels
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Load and transform images
        batch_size = len(img_path)
        images = torch.zeros(batch_size, 3, img_size, img_size, dtype=torch.float32)
        for i in range(batch_size):
            # image = cv2.imread(img_path[i]).numpy()
            image = Image.open(img_path[i]).convert("RGB")
            # images[i] = image
            images[i] = train_transform(image)  # Assuming train_transform is defined elsewhere
        # Forward pass
        # images = images.to(device)
        output1,output2 = model(images, inputs)

        # Backward and optimize
        optimizer.zero_grad()

        loss1 = criterion1(labels, output1)
        loss2 = criterion2(labels_aux, output2)
         
        loss = loss1+loss2

        if loss1 <= 1000:
            
            loss.backward()

            optimizer.step()
            scheduler.step()

            total_loss += loss1.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Training Loss: {avg_loss:.4f}")

    return avg_loss

In [111]:
def evaluate_epoch(test_loader,model,criterion1,criterion2,device,img_size):
    model.eval()
    total_loss = 0
    for i, data in tqdm(enumerate(test_loader)):
        # Unpack data
        img_path, inputs, labels, labels_aux = data

        # Prepare image paths
        img_path = ['/Users/zhangyumeng/Desktop/23Winter/273P/project/planttraits2024/'+i for i in img_path]

        # # Prepare inputs and labels
        inputs = inputs.to(device)
        labels = labels.to(device)

        batch_size = len(img_path)
        images = torch.zeros(batch_size, 3, img_size, img_size, dtype=torch.float32)

        for i in range(batch_size):
            # image = cv2.imread(img_path[i]).numpy()
            image = Image.open(img_path[i]).convert("RGB")
            images[i] = train_transform(image)  # Assuming train_transform is defined elsewhere
        # Forward pass
        # images = images.to(device)
        output1,output2 = model(images, inputs)
        loss1 = criterion1(output1, labels)       
        loss2 = criterion2(output2,labels_aux)
        print('loss1',loss1)
        print('loss2',loss2)
        loss = loss1+loss2
        total_loss += loss
    avg_loss = total_loss/len(test_loader)
    return avg_loss
    

In [150]:
# model.to(device)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Combine_model()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=weight_decay, lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                T_max=MAX_EPOCH,
                                                eta_min=0,
                                                last_epoch=-1)
criterion1 = MSELoss(use_mask=False)
criterion2 = MSELoss(use_mask=True)

for _ in range(MAX_EPOCH):
    print('epoch',_)
    train_epoch(train_loader,model,criterion1,criterion2,optimizer,scheduler,device,img_size)
    # evaluate_epoch(val_loader,model, criterion1,criterion2, device, img_size)

/Users/zhangyumeng/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/zhangyumeng/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


epoch 0


200it [00:21,  9.16it/s]


Training Loss: 6.4789
epoch 1


200it [00:20,  9.53it/s]


...

Training Loss: 6.2042
epoch 3


200it [00:20,  9.61it/s]


Training Loss: 6.1531
epoch 4


200it [00:20,  9.59it/s]

Training Loss: 6.1366


In [114]:
model = Combine_model()
criterion1 = R2Loss(use_mask=False)
criterion2 = R2Loss(use_mask=True)
optimizer = torch.optim.Adam(model.parameters(), weight_decay=weight_decay, lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                T_max=MAX_EPOCH,
                                                eta_min=0,
                                                last_epoch=-1)
for _ in range(MAX_EPOCH):
    evaluate_epoch(val_loader,model, criterion1,criterion2, device, img_size)

/Users/zhangyumeng/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/zhangyumeng/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
7it [00:00, 30.19it/s]

loss1 tensor(1265.7909, grad_fn=<MeanBackward0>)
loss2 tensor(1854.7197, grad_fn=<MeanBackward0>)
loss1 tensor(13.4011, grad_fn=<MeanBackward0>)
loss2 tensor(191.0170, grad_fn=<MeanBackward0>)
loss1 tensor(4414.2993, grad_fn=<MeanBackward0>)
loss2 tensor(27.7515, grad_fn=<MeanBackward0>)
loss1 tensor(7.2002, grad_fn=<MeanBackward0>)
loss2 tensor(29.5333, grad_fn=<MeanBackward0>)
loss1 tensor(678.4037, grad_fn=<MeanBackward0>)
loss2 tensor(1646.3619, grad_fn=<MeanBackward0>)
loss1 tensor(2650.4602, grad_fn=<MeanBackward0>)
loss2 tensor(190.2494, grad_fn=<MeanBackward0>)
loss1 tensor(114.4610, grad_fn=<MeanBackward0>)
loss2 tensor(42.9691, grad_fn=<MeanBackward0>)


11it [00:00, 29.87it/s]

loss1 tensor(37233.1992, grad_fn=<MeanBackward0>)
loss2 tensor(46785.9883, grad_fn=<MeanBackward0>)
loss1 tensor(22.9265, grad_fn=<MeanBackward0>)
loss2 tensor(69.5529, grad_fn=<MeanBackward0>)
loss1 tensor(460.8175, grad_fn=<MeanBackward0>)
loss2 tensor(1409.1577, grad_fn=<MeanBackward0>)
loss1 tensor(909.8994, grad_fn=<MeanBackward0>)
loss2 tensor(6805.1777, grad_fn=<MeanBackward0>)
loss1 tensor(9224.2285, grad_fn=<MeanBackward0>)
loss2 tensor(17.6979, grad_fn=<MeanBackward0>)


18it [00:00, 28.44it/s]

...

153it [00:04, 30.66it/s]

loss1 tensor(273.7056, grad_fn=<MeanBackward0>)
loss2 tensor(40.5606, grad_fn=<MeanBackward0>)
loss1 tensor(89.8220, grad_fn=<MeanBackward0>)
loss2 tensor(6.2711, grad_fn=<MeanBackward0>)
loss1 tensor(383.9755, grad_fn=<MeanBackward0>)
loss2 tensor(695.2009, grad_fn=<MeanBackward0>)
loss1 tensor(144.2856, grad_fn=<MeanBackward0>)
loss2 tensor(1328.4877, grad_fn=<MeanBackward0>)
loss1 tensor(47.2234, grad_fn=<MeanBackward0>)
loss2 tensor(57.7886, grad_fn=<MeanBackward0>)
loss1 tensor(169.0717, grad_fn=<MeanBackward0>)
loss2 tensor(1699.9108, grad_fn=<MeanBackward0>)
loss1 tensor(737.5786, grad_fn=<MeanBackward0>)
loss2 tensor(56.1127, grad_fn=<MeanBackward0>)


160it [00:05, 30.98it/s]

loss1 tensor(38.8151, grad_fn=<MeanBackward0>)
loss2 tensor(464.4643, grad_fn=<MeanBackward0>)
loss1 tensor(70.6085, grad_fn=<MeanBackward0>)
loss2 tensor(81.6964, grad_fn=<MeanBackward0>)
loss1 tensor(1437.2954, grad_fn=<MeanBackward0>)
loss2 tensor(11318.5146, grad_fn=<MeanBackward0>)
loss1 tensor(24.6140, grad_fn=<MeanBackward0>)
loss2 tensor(197.2419, grad_fn=<MeanBackward0>)
loss1 tensor(170.4754, grad_fn=<MeanBackward0>)
loss2 tensor(103.1587, grad_fn=<MeanBackward0>)
loss1 tensor(1700.4144, grad_fn=<MeanBackward0>)
loss2 tensor(35.1899, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 